In [ ]:
###########################
# SECTION 1: TENSORFLOW / KERAS
###########################

# Imports
import os
import numpy as np
import random
import shutil
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Placeholder paths (change during exam)
DATA_DIR = '/RANDOM_PATH/IMAGES'
NPZ_PATH = '/RANDOM_PATH/dataset.npz'  # for .npz/.npy examples
OUTPUT_DIR = '/RANDOM_PATH/OUTPUT'

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ----------------------
# DATA LOADING METHODS
# ----------------------

# 1) Load from .npz / .npy
def load_from_npz(npz_path=NPZ_PATH):
    data = np.load(npz_path)
    # expecting keys: X_train, y_train, X_val, y_val, X_test, y_test
    return data

# 2) image_dataset_from_directory (folder structure class-per-folder)
def load_with_image_dataset_from_directory(root_dir=DATA_DIR, image_size=(224,224), batch_size=32):
    ds = tf.keras.utils.image_dataset_from_directory(
        root_dir,
        image_size=image_size,
        batch_size=batch_size
    )
    return ds

# 3) load_img + manual batching (single image)
def load_single_image(path, target_size=(224,224)):
    img = load_img(path, target_size=target_size)
    arr = img_to_array(img) / 255.0
    return arr

# 4) ImageDataGenerator (flow_from_directory)

def get_imagedatagenerator(root_dir, target_size=(224,224), batch_size=32):
    datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2)
    gen = datagen.flow_from_directory(root_dir, target_size=target_size, batch_size=batch_size, class_mode='categorical')
    return gen

# 5) tf.data custom pipeline

def tfdata_from_filelist(file_list, labels=None, image_size=(224,224), batch_size=32, shuffle=True):
    def _parse(path, label=None):
        img = tf.io.read_file(path)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.resize(img, image_size)
        img = tf.cast(img, tf.float32) / 255.0
        return (img, label) if label is not None else img

    ds = tf.data.Dataset.from_tensor_slices((file_list, labels)) if labels is not None else tf.data.Dataset.from_tensor_slices(file_list)
    if labels is not None:
        ds = ds.map(lambda p, l: _parse(p, l), num_parallel_calls=tf.data.AUTOTUNE)
    else:
        ds = ds.map(lambda p: _parse(p), num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=1024)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# 6) Manual split when all images in single folder (no labels)

def split_single_folder_to_train_val_test(single_folder, out_dir=OUTPUT_DIR, val_frac=0.1, test_frac=0.1, seed=123):
    # Assumes images are named and labels will be assigned externally
    imgs = [os.path.join(single_folder, f) for f in os.listdir(single_folder) if f.lower().endswith(('.png','.jpg','.jpeg'))]
    train_and_val, test = train_test_split(imgs, test_size=test_frac, random_state=seed)
    train, val = train_test_split(train_and_val, test_size=val_frac/(1-test_frac), random_state=seed)
    # Create folders
    for split, files in zip(['train','val','test'], [train, val, test]):
        target = os.path.join(out_dir, split)
        os.makedirs(target, exist_ok=True)
        for p in files:
            shutil.copy(p, target)
    return {'train':train, 'val':val, 'test':test}

# 7) Using CSV file with paths/labels

def load_from_csv(csv_path):
    import pandas as pd
    df = pd.read_csv(csv_path)
    # expects columns: filepath, label
    X = df['filepath'].values
    y = df['label'].values
    return X, y

# 8) TFRecord example writer/reader (skeleton)
def write_tfrecord_example(image_paths, labels, output_tfrecord):
    with tf.io.TFRecordWriter(output_tfrecord) as writer:
        for p, l in zip(image_paths, labels):
            img = open(p, 'rb').read()
            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img])),
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(l)]))
            }
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

# ----------------------
# Experiment skeletons (TF)
# ----------------------

# Ex 1: DNN for image classification (on flattened images)
def experiment_dnn_tf(X_train, y_train, X_val, y_val, input_shape=(32*32*3,), n_classes=10):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(n_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)
    return model

# Ex 2: CNN vs DNN comparison

def experiment_cnn_tf(train_ds, val_ds, n_classes=10):
    model = keras.Sequential([
        layers.Input(shape=(224,224,3)),
        layers.Conv2D(32,3,activation='relu'),
        layers.MaxPool2D(),
        layers.Conv2D(64,3,activation='relu'),
        layers.MaxPool2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(n_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_ds, validation_data=val_ds, epochs=10)
    return model

# Ex 3: Object detector (simplified: predict bounding box + class)
def experiment_object_detection_tf(train_ds_with_boxes, val_ds_with_boxes):
    # train_ds_with_boxes should yield (image, [x_min,y_min,x_max,y_max,class_id])
    inputs = layers.Input(shape=(224,224,3))
    x = layers.Conv2D(32,3,activation='relu')(inputs)
    x = layers.MaxPool2D()(x)
    x = layers.Flatten()(x)
    bbox = layers.Dense(4, activation='sigmoid', name='bbox')(x)
    cls = layers.Dense(1, activation='sigmoid', name='class')(x)
    model = keras.Model(inputs=inputs, outputs=[bbox, cls])
    model.compile(optimizer='adam', loss={'bbox':'mse','class':'binary_crossentropy'})
    model.fit(train_ds_with_boxes, validation_data=val_ds_with_boxes, epochs=10)
    return model

# Ex 4: FCN-like segmentation (skeleton)
def experiment_segmentation_tf(train_ds, val_ds, n_classes=2):
    inputs = layers.Input(shape=(128,128,3))
    c1 = layers.Conv2D(32,3,activation='relu', padding='same')(inputs)
    p1 = layers.MaxPool2D()(c1)
    c2 = layers.Conv2D(64,3,activation='relu', padding='same')(p1)
    up = layers.UpSampling2D()(c2)
    outputs = layers.Conv2D(n_classes, 1, activation='softmax')(up)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_ds, validation_data=val_ds, epochs=10)
    return model

# Ex 5 & 6: Autoencoder and Conv Autoencoder

def experiment_autoencoder_tf(x_train, x_val, latent_dim=64):
    input_img = layers.Input(shape=(28,28,1))
    x = layers.Flatten()(input_img)
    encoded = layers.Dense(latent_dim, activation='relu')(x)
    decoded = layers.Dense(28*28, activation='sigmoid')(encoded)
    decoded = layers.Reshape((28,28,1))(decoded)
    auto = keras.Model(input_img, decoded)
    auto.compile(optimizer='adam', loss='mse')
    auto.fit(x_train, x_train, validation_data=(x_val, x_val), epochs=10)
    return auto

# Ex 7: Denoising autoencoder (skeleton)
def experiment_denoising_autoencoder_tf(x_train, x_val, noise_factor=0.5):
    x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
    x_val_noisy = x_val + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_val.shape)
    return experiment_autoencoder_tf(x_train_noisy, x_val_noisy)

# Ex 8: Image captioning skeleton (feature extractor + RNN decoder)

def experiment_image_captioning_tf(image_features, captions_seq, tokenizer, vocab_size):
    # image_features: precomputed features per image
    # captions_seq: tokenized sequences
    img_input = layers.Input(shape=(image_features.shape[1],))
    seq_input = layers.Input(shape=(None,))
    se = layers.Embedding(vocab_size, 128)(seq_input)
    se = layers.LSTM(256)(se)
    concat = layers.concatenate([img_input, se])
    out = layers.Dense(vocab_size, activation='softmax')(concat)
    model = keras.Model([img_input, seq_input], out)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    # model.fit([...])
    return model

# Ex 9 & 10: LSTM handwriting recognition / RNN vs LSTM vs GRU for time series

def experiment_rnn_tf(X_train, y_train, cell='LSTM'):
    model = keras.Sequential()
    model.add(layers.Input(shape=X_train.shape[1:]))
    if cell == 'LSTM':
        model.add(layers.LSTM(64))
    elif cell == 'GRU':
        model.add(layers.GRU(64))
    else:
        model.add(layers.SimpleRNN(64))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=10)
    return model

# Ex 11: GAN skeleton (generator/discriminator)

def experiment_gan_tf(latent_dim=100):
    # Generator
    gen_in = layers.Input(shape=(latent_dim,))
    x = layers.Dense(7*7*128, activation='relu')(gen_in)
    x = layers.Reshape((7,7,128))(x)
    x = layers.UpSampling2D()(x)
    gen_out = layers.Conv2D(1, kernel_size=3, padding='same', activation='sigmoid')(x)
    generator = keras.Model(gen_in, gen_out)

    # Discriminator
    dis_in = layers.Input(shape=(28,28,1))
    y = layers.Conv2D(64,3, strides=2, padding='same', activation='relu')(dis_in)
    y = layers.Flatten()(y)
    dis_out = layers.Dense(1, activation='sigmoid')(y)
    discriminator = keras.Model(dis_in, dis_out)
    discriminator.compile(optimizer='adam', loss='binary_crossentropy')

    # GAN combined (simple)
    discriminator.trainable = False
    z = layers.Input(shape=(latent_dim,))
    img = generator(z)
    validity = discriminator(img)
    combined = keras.Model(z, validity)
    combined.compile(optimizer='adam', loss='binary_crossentropy')
    return generator, discriminator, combined

# Ex 12: Reinforcement Learning skeleton (policy gradient) - pseudocode

def experiment_rl_policy_gradient(env):
    # env : any OpenAI Gym-like environment (offline exam: provide a mock env)
    pass

###########################
# SECTION 2: PYTORCH
###########################

# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from PIL import Image

# Placeholder paths
PT_DATA_DIR = '/RANDOM_PATH/IMAGES'
PT_NPZ = '/RANDOM_PATH/dataset.npz'

# ----------------------
# DATA LOADING METHODS (PyTorch)
# ----------------------

# 1) datasets.ImageFolder

def get_imagefolder_loader(root_dir=PT_DATA_DIR, batch_size=32, image_size=(224,224)):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])
    dataset = datasets.ImageFolder(root=root_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

# 2) Custom Dataset from folder or CSV
class CustomImageDataset(Dataset):
    def __init__(self, img_paths, labels=None, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        p = self.img_paths[idx]
        img = Image.open(p).convert('RGB')
        if self.transform:
            img = self.transform(img)
        if self.labels is not None:
            return img, self.labels[idx]
        return img

# 3) Loading .npz / .npy into tensors

def load_npz_pytorch(npz_path=PT_NPZ):
    data = np.load(npz_path)
    X = torch.tensor(data['X']).float()
    y = torch.tensor(data['y']).long()
    return X, y

# 4) Manual train/val/test split from single folder

def split_folder_pytorch(single_folder, out_dir='/tmp/pytorch_split', val_frac=0.1, test_frac=0.1, seed=123):
    os.makedirs(out_dir, exist_ok=True)
    imgs = [os.path.join(single_folder,f) for f in os.listdir(single_folder) if f.lower().endswith(('.png','.jpg','.jpeg'))]
    train_and_val, test = train_test_split(imgs, test_size=test_frac, random_state=seed)
    train, val = train_test_split(train_and_val, test_size=val_frac/(1-test_frac), random_state=seed)
    for split, files in zip(['train','val','test'], [train,val,test]):
        d = os.path.join(out_dir, split)
        os.makedirs(d, exist_ok=True)
        for p in files:
            shutil.copy(p, d)
    return {'train':train, 'val':val, 'test':test}

# ----------------------
# Experiment skeletons (PyTorch)
# ----------------------

# Ex 1: DNN
class SimpleDNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Ex 2: CNN
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,3,padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,3,padding=1)
        self.fc1 = nn.Linear(64*56*56, 128)
        self.fc2 = nn.Linear(128, num_classes)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Ex 3: Object detection skeleton (predict bbox + class)
class TinyDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3,16,3,padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(16*112*112, 128)
        self.bbox = nn.Linear(128, 4)
        self.cls = nn.Linear(128, 1)
    def forward(self,x):
        x = self.pool(F.relu(self.conv(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc(x))
        return self.bbox(x), torch.sigmoid(self.cls(x))

# Ex 4: Segmentation skeleton (simple conv-deconv)
class TinySeg(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.enc = nn.Sequential(nn.Conv2d(3,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2))
        self.dec = nn.Sequential(nn.Upsample(scale_factor=2), nn.Conv2d(16,num_classes,3,padding=1))
    def forward(self,x):
        x = self.enc(x)
        x = self.dec(x)
        return x

# Ex 5 & 6: Autoencoder
class AutoencoderPT(nn.Module):
    def __init__(self, latent_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(nn.Flatten(), nn.Linear(28*28, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 28*28), nn.Unflatten(1, (1,28,28)))
    def forward(self,x):
        z = self.encoder(x)
        return self.decoder(z)

# Ex 7: Denoising AE uses same model + noisy inputs

# Ex 8: Image captioning skeleton (CNN features + RNN decoder)
class CaptionDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    def forward(self, captions):
        x = self.embed(captions)
        out, _ = self.lstm(x)
        return self.fc(out)

# Ex 9 & 10: RNN/LSTM/GRU for sequences
class SequenceModelPT(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, cell='LSTM'):
        super().__init__()
        if cell == 'LSTM':
            self.rnn = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        elif cell == 'GRU':
            self.rnn = nn.GRU(input_dim, hidden_dim, batch_first=True)
        else:
            self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
    def forward(self,x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        return self.fc(out)

# Ex 11: GAN skeleton (PyTorch)
class GeneratorPT(nn.Module):
    def __init__(self, latent_dim=100):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128*7*7),
            nn.ReLU(),
            nn.Unflatten(1, (128,7,7)),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128,1,3,padding=1),
            nn.Sigmoid()
        )
    def forward(self,z):
        return self.model(z)

class DiscriminatorPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1,64,3,stride=2,padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*14*14,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.model(x)

# Ex 12: RL skeleton (PyTorch)

def rl_policy_gradient_skeleton():
    # For offline lab: implement a mock environment or simple bandit
    pass